# web scraping RIP.ie with beautifulsoup

In [2]:
import time
import pandas as pd
from bs4 import BeautifulSoup
import requests
import datefinder

def get_dn_page(n):
    """Get death notice text from page matching the id number"""
    
    url = 'https://rip.ie/showdn.php?dn=%s' %n
    req = requests.get(url)
    soup = BeautifulSoup(req.content, 'html.parser')
    if soup.title == None:
        title = ''
    else:        
        title  = soup.title.text.strip()
    name=''    
    for s in ['Death Notice of ','The death has occurred of ']:
        if title.startswith(s):
            name = title.split(s)[1]   
    elem = soup.find_all("div", id="dn_photo_and_text")
    
    if len(elem) == 0:
        return name, '', '', '', ''
    rows = elem[0].find_all('p')
    if len(rows) == 0:
        rows = elem[0].find_all('td')
    text = ';'.join([r.text.strip() for r in rows]).replace('\n','')
    #address
    addrelem = soup.find("span", class_='small_addr') 
    if addrelem != None:
        address = addrelem.text.strip()
    else:
        address = ''
    #county  
    ctyelem = soup.find("li", class_='fd_county') 
    if ctyelem != None:
        county = ctyelem.text.strip()
    else:
        county = ''
    #date
    dateelem = soup.find("div", class_='ddeath')
    if dateelem == None:
        dateelem = soup.find("div", class_='dpubl')
    s = dateelem.text.strip()
    try:
        date = list(datefinder.find_dates(s))[0]
    except:
        date = ''
    print (n, date, name, address, county)
    return name, date, county, address, text

In [2]:
get_dn_page(390045)

390045 2019-06-05 00:00:00 Margaret  Kelly Avenue Grove, Ballymodan Place, Bandon,  Cork Cork


('Margaret  Kelly',
 datetime.datetime(2019, 6, 5, 0, 0),
 'Cork',
 'Avenue Grove, Ballymodan Place, Bandon,  Cork',
 "Kelly (Avenue Grove, Ballymodan Place, Bandon and late of Currivreeda West) on June 5th 2019. Margaret, beloved daughter of the late Timothy and Ellen. Sadly missed by her loving sisters Ann and Eileen, brothers Teddy and John, Margaret's Partner William, brothers-in-law, sisters-in-law, nieces, nephews, relatives and good friend Malcolm.;;Rosary on Friday evening at 7pm in St. Patrick's Church, Bandon. Requiem Mass on Saturday at 12 noon, funeral afterwards to the adjoining cemetery.;;May Margaret Rest in Peace")

In [28]:
#df = pd.read_csv('rip_dn_scrape.csv')
df = pd.read_pickle('rip_dn_scrape.pkl')
len(df)

473362

In [34]:
z.to_csv('/home/damien/Downloads/test_day.csv')

## iterate over a range of ids and get info

In [ ]:
#read current table in so we skip those already done
df = pd.read_pickle('rip_dn_scrape.pkl')
print (len(df))
ids = list(df.id)

results={}
for n in range(470000,473370):
    if n in ids:
        continue
    name,date,cty,addr,txt = get_dn_page(n)    
    results[n] = [name,date,cty,addr,txt]
    time.sleep(0.05)

In [63]:
res = pd.DataFrame.from_dict(results,orient='index',columns=['name','date','county','address','notice']).reset_index()
res = res.rename(columns={'index':'id'})

In [64]:
res

,id,name,date,county,address,notice
0,473362,Kieran Timlin,2021-10-20,Fermanagh,"4 The Meadows, Irvinestown, Fermanagh","Kieran Timlin R.I.P, 19th October 2021 (4 The ..."
1,473363,Hugh O'BYRNE,2021-10-19,Carlow,"Ballykealy, Ballon, Carlow","Hugh O’Byrne, October 19th 2021 (peacefully) a..."
2,473364,Christy (Kit) Kelly,2021-10-19,Meath,"Castletown, Athboy, Meath","The death has occurred of Christy (Kit) Kelly,..."
3,473365,PJ McLoughlin,2021-10-19,Leitrim,"Sheena, Drumkeerin, Leitrim","The death has occurred of PJ McLoughlin, Sheen..."
4,473366,Ian Pearson,2021-10-17,Meath,"Connaught Place, Athboy, Meath","The death has occurred of Ian Pearson, Connaug..."
5,473367,John A. Nolan,2021-10-19,Kerry,"2 St. Stephen's Park, Castleisland, Kerry","John A.Nolan 2 St. Stephen's Park, Castleislan..."
6,473368,Michael (Mikie) Bourke,2021-10-11,Limerick,"Ballyhobin, Caherconlish, Limerick / Shannon,...","London and formerly of Ballyhobin, Caherconlis..."
7,473369,Kate (Kitty) Barry (née Hurley),2021-10-19,,"Curraglass, Cork / Tallow, Waterford","Barry, Glasshouse, Curraglass & late of Clonak..."


In [65]:
x = pd.concat([df,res]).reset_index(drop=True)
x=x[~x.id.duplicated(keep='first')]
print (len(df),len(res),len(x))
x.to_pickle('rip_dn_scrape.pkl')

473362 8 473370


In [66]:
print (len(x))
x=x.replace('',None).dropna(axis=0,subset=['date'])
x['date'] = pd.to_datetime(x['date']).apply(lambda x: x.strftime('%d/%m/%Y'))
x=x.drop_duplicates(['name','notice'])
x=x.drop_duplicates(['name','address'])
x=x.drop_duplicates(['name','date','county'])
x=x.sort_values('id')
print (len(x))
#x.to_csv('rip_dn_scrape_processed.csv')
x.to_pickle('rip_dn_scrape_processed.pkl')

473370
453636
